In [1]:
np.set_printoptions(linewidth=400, precision=3)
%run ../Utilities/Utilities.ipynb
%matplotlib qt

## Euler solver

In [2]:
def EulerSolver(m, fnc, n=None, returnLast = True, returnForPlot=False):
    '''
    m: number of points in xaxis direction
    n: number of points in time. Is equal to 300*(int(np.ceil(np.sqrt((m+1))))-1) if no input is given
    fnc: is the boundary function.
    returnLast: Solution returns only the last vector. 
    returnForPlot: Makes it so the haxis is also returned
    
    returns: axis, solution
    '''
    #Make a default value for n if none is given
    if n==None:
        n=300*(int(np.ceil(np.sqrt((m+1))))-1)
        
    #Step sizes
    h = 2/(m+1)
    k = 1/(n+1)
    r = (1+np.pi**2)
    
    #Make the initial distribution in x
    xaxis = np.arange(-1,1+h,h)[0:m+1]
    haxis = np.arange(0,1+k,k)[0:n+2]
    U0 = fnc(xaxis)
    
    #Make solution matrix
    solution = np.zeros((n+2,m+1))
    solution[0,:] = U0
    
    #Make diagonal
    diag = -np.zeros(m+1)
    
    #Make superdiagonal
    sup = (r/(2*h)-3/(8*h**3))
    
    #Make subdiagonal
    sub = -sup
    
    #Make subsubdiagonal
    subsub = -1/(8*h**3)
    
    #Make supsupdiagonal
    supsup = 1/(8*h**3)
    
    #Make supsupsupdiagonal
    supsupsup = (-1/(8*h**3))
    
    #Make subsubsubdiagonal
    subsubsub = -supsupsup
    
    #Make system of equations
    A = diags([diag,sup,sub,supsup,subsub,supsupsup,subsubsub],[0,1,-1,3,-3,m-2,-(m-2)]).toarray()
    A[0,-1] = 3/(8*h**3) - r/(2*h)
    A[-1,0] = -3/(8*h**3) + r/(2*h)
    
    #Iterate in time
    for i in range(1,n+2):
        solution[i,:] = solution[i-1,:] - k*A@solution[i-1,:]
    
    #Return only last value
    if returnLast:
        returnSolution = solution[-1,:]
    else: #Return whole solution
        returnSolution = solution.transpose()
    
    #Return an additional axis in time. 
    if returnForPlot:
        return xaxis, haxis, returnSolution
    
    return xaxis, returnSolution
    
    

## Crank nicolson solver

In [3]:
def CrankSolver(m, fnc, n=None, returnLast = True, returnForPlot=False):
    '''
    m: number of points in xaxis direction
    n: number of points in time. Is equal to 300*(int(np.ceil(np.sqrt((m+1))))-1) if no input is given
    fnc: is the boundary function.
    returnLast: Solution returns only the last vector. 
    returnForPlot: Makes it so the haxis is also returned
    
    returns: axis, solution
    '''
    
    #Make a default value for n if none is given
    if n==None:
        n=300*(int(np.ceil(np.sqrt((m+1))))-1)
        
    #Step sizes
    h = 2/(m+1)
    k = 1/(n+1)
    r = k/h**2
    
    #Make the initial distribution in x
    xaxis = np.arange(-1,1+h,h)[0:m+1]
    haxis = np.arange(0,1+k,k)[0:n+2]
    U0 = fnc(xaxis)
    
    #Term for simplification
    w = r*(4*h**2*(1+np.pi**2)-3)/(16*h)
    
    #Make some values that are on diagonals
    diag = np.ones(m+1)  #For the diags fnc, at least one element needs to be a vector
    sup = w    
    sub = -w
    
    #Make supsup diagonal and subsub diagonal
    supsup = -r/(16*h)
    subsub = r/(16*h)
    
    #Make supsupsup diagonal and subsubsubdiagonal
    supsupsup = r/(16*h)
    subsubsub = -r/(16*h)
    
    #Make the A-matrix
    A = diags([diag,sup,sub,supsup,subsub,supsupsup,subsubsub],[0,1,-1,m-2,-(m-2),3,-3]).todense() #Full matrix
    A[0,m] = -w
    A[m,0] = w
    A=sp.csc_matrix(A) #Make the matrix sparse
    
    #Make the B-matrix
    B = diags([diag,-sup,-sub,-supsup,-subsub,-supsupsup,-subsubsub],[0,1,-1,m-2,-(m-2),3,-3]).todense() #Full matrix
    B[0,m] = w
    B[m,0] = -w
    B = sp.csc_matrix(B) #Make the matrix sparse
    
    #Make solution matrix
    solution = np.zeros((n+2,m+1))
    solution[0,:] = U0
    
    #Lu factor and use lu_solve for faster computing time
    splu = sl.splu(A)
    
    
    #Iterate in time
    for i in range(1,n+2):
        c = B*solution[i-1, :]
        solution[i, :] = splu.solve(c)
    
    #Return only last value
    if returnLast:
        returnSolution = solution[-1,:]
    else: 
        returnSolution = solution.transpose()
    
    #Return an additional axis in time. 
    if returnForPlot:
        return xaxis, haxis, returnSolution
    
    return xaxis, returnSolution

## Boundary condition functions

In [4]:
@jit
def boundary1(x): #Boundary condition
    return np.sin(np.pi*x)

@jit #Boundary condition for optional 1
def boundary2(x):
    return np.cos(np.pi*x)

@jit
def boundary3(x): #Boundary condition for optinal 2
    return np.cos(np.pi*x) - 2*np.sin(np.pi*x)

@jit
def solution1(x, t=1): #Solution
    return np.sin(np.pi*(x-t))

@jit
def solution2(x, t=1): #Solution for optional 1
    return np.cos(np.pi*(x-t))

@jit
def solution3(x, t=1): #Solution for optional 2
    return np.cos(np.pi*(x-t)) - 2*np.sin(np.pi*(x-t))

## L2-norm plot

In [3]:
def plotL2(SolutionMatrix,haxis):
    '''
    Desription: Plot the l2-norm over time.
    Soltuion: A nxm matrix, for n time steps and m steps in space.
    haxis: A n-vector with time points
    '''
    Solution = SolutionMatrix.transpose()
    #Assign empty vector
    l2 = np.zeros(len(Solution))
    
    #Calcualte the L2-norm for every time step of the solution
    for i in range(len(l2)):
        l2[i] = norm(Solution[i])
        
    #Plot the values. Also plot the mean
    plt.plot(haxis,l2, label = "$\ell_2$-Norm") #Plottin L2 norm
    plt.plot(haxis, np.ones(len(l2))*np.mean(l2), label = "Mean value")
    plt.xlabel("t")
    plt.ylabel("$||u(x,t)||_{\ell_2}$")
    plt.legend()
    plt.show()
    print("Mean value:",np.mean(l2))
    print("Standard deviance", np.std(l2))
    